## 신경망의 톱니바퀴: 텐서 연산
* 심층 신경망이 학습한 모든 변환을 수치 데이터 텐서에 적용하는 몇 종류의 **텐서 연산**(tensor operation)으로 나타낼 수 있다.
* `keras.layers.Dense(512, activation='relu')`
* 이 층은 2D 텐서를 입력으로 받고 입력 텐서의 새로운 표현인 또 다른 2D 텐서를 반환하는 함수처럼 해석할 수 있다.
* `output = relu(dot(W, input) + b)`
* (W는 2D 텐서, b는 벡터)

### 원소별 연산
* relu 함수와 덧셈은 원소별 연산(element-wise operation).
* 텐서에 있는 각 원소에 독립적으로 적용.

In [1]:
def naive_relu(x):
    assert len(x.shape) == 2

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

* 넘파이 배열을 다룰 때는 최적화된 넘파이 내장 함수로 이런 연산들들 처리할 수 있다.
* 넘파이는 시스템에 설치된 BLAS(Basic Linear Algebra Subprogram) 구현에 복잡한 일들을 위임한다.
* BLAS는 고도로 병렬화되고 효율적인 저수준의 텐서 조작 루틴이며, 전형적으로 포트란(Fortran)이나 C언어로 구현.

### 브도르캐스팅
* 크기가 다른 두 텐서가 더해질 때 모호하지 않고 실행 가능하다면 작은 텐서가 큰 텐서의 크기에 맟추어 브도르캐스팅(broadcasting)된다.
    1. 큰 텐서의 ndim에 맞도록 작은 텐서에 (브도르캐스팅 축이라고 부르는) 축이 추가된다.
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복된다.
* 구현 입장에서는 새로운 텐서가 만들어지면 매우 비효율적
* 메모리 수준이 아니라 알고리즘 수준에서 일어난다.

In [2]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[u, j] += y[j]
    return x

### 텐서 점곱 (tensor product)

In [3]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]

    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

In [4]:
import numpy as np

def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]

    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

In [5]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

In [6]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]

    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

### 텐서 크기 변환 (tensor reshaping)
* `train_images = train-images.reshape((60000, 28 * 28))`

In [7]:
x = np.array([
    [0., 1.],
    [2., 3.],
    [4., 5.]
])
print(x.shape)

(3, 2)


In [8]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [9]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

* 자주 사용하는 특별한 트기 변환은 전치(transposition)이다.
* 행렬의 전치는 행과 열을 바꾸는 것을 의미.

### 텐서 연산의 기하학적 해석
* 텐서 연산이 조작하는 텐서의 내용은 어떤 기하학적 공간에 있는 좌표 포인트로 해석될 수 있기 때문에 모든 텐서 연산은 기하학적 해석이 가능하다.
### 딥러닝의 기하학적 해석